# MCP

In [8]:
import os

import chromadb
import dotenv
from agents import Agent, Runner, function_tool, trace
from agents.mcp import MCPServerStreamableHttp

dotenv.load_dotenv()

True

Let's set up our RAG database connection:

In [9]:
chroma_client = chromadb.PersistentClient(path="../chroma")
nutrition_db = chroma_client.get_collection(name="nutrition_db")

In [10]:
# This is the same code as in the rag.ipynb notebook


@function_tool
def calorie_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function for a RAG database to look up calorie information for specific food items, but not for meals.

    Args:
        query: The food item to look up.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the nutrition information.
    """

    results = nutrition_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        metadata = results["metadatas"][0][i]
        food_item = metadata["food_item"].title()
        calories = metadata["calories_per_100g"]
        category = metadata["food_category"].title()

        formatted_results.append(
            f"{food_item} ({category}): {calories} calories per 100g"
        )

    return "Nutrition Information:\n" + "\n".join(formatted_results)

Integrate EXA Search as an MCP:

In [11]:
exa_search_mcp = MCPServerStreamableHttp(
    name="Exa Search MCP",
    params={
        "url": f"https://mcp.exa.ai/mcp?exaApiKey={os.environ.get("EXA_API_KEY")}",
        "timeout": 30,
    },
    client_session_timeout_seconds=30,
    cache_tools_list=True,
    max_retry_attempts=1,
)

await exa_search_mcp.connect()

In [12]:
from agents import Agent, Runner, WebSearchTool

agent = Agent(
    name="Assistant",
    tools=[WebSearchTool(),],
    mcp_servers=[exa_search_mcp],
)

In [13]:
# Exa Search MCP code comes here:

calorie_agent_with_search = Agent(
    name="Nutrition Assistant",
    instructions="""
    * You are a helpful nutrition assistant giving out calorie information.
    * You give concise answers.
    * You follow this workflow:
        0) First, use the calorie_lookup_tool to get the calorie information of the ingredients. But only use the result if it's explicitly for the food requested in the query.
        1) If you couldn't find the exact match for the food or you need to look up the ingredients, search the EXA web to figure out the exact ingredients of the meal.
        Even if you have the calories in the web search response, you should still use the calorie_lookup_tool to get the calorie
        information of the ingredients to make sure the information you provide is consistent.
        2) Then, if necessary, use the calorie_lookup_tool to get the calorie information of the ingredients.
    * Even if you know the recipe of the meal, always use Exa Search to find the exact recipe and ingredients.
    * Once you know the ingredients, use the calorie_lookup_tool to get the calorie information of the individual ingredients.
    * If the query is about the meal, in your final output give a list of ingredients with their quantities and calories for a single serving. Also display the total calories.
    * Don't use the calorie_lookup_tool more than 10 times.
    """,
    tools=[calorie_lookup_tool],
     mcp_servers=[exa_search_mcp],
)

Reference query - shouldn't use ExaSearch:

In [14]:
with trace("Nutrition Assistant with MCP - Only uses calorie_lookup_tool"):
    result = await Runner.run(
        agent,
        "How many calories are in total in a banana and an apple? Also give calories per 100g",
    )
    print(result)


RunResult:
- Last agent: Agent(name="Assistant", ...)
- Final output (str):
    - Calories per 100 g: banana ~89 kcal; apple ~52 kcal.
    - Typical whole fruits (rough estimates): medium banana (~118 g) ≈ 105 kcal; medium apple (~182 g) ≈ 95 kcal.
    - Total for one banana + one apple: about 200 kcal (roughly 200 kcal, depending on fruit sizes).
    
    If you have exact weights for the banana and apple, I can calculate a precise total.
- 2 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


In [15]:
with trace("Nutrition Assistant with MCP - Only uses calorie_lookup_tool"):
    result = await Runner.run(
        calorie_agent_with_search,
        "How many calories are in total in a banana and an apple? Also give calories per 100g",
    )
    print(result)

RunResult:
- Last agent: Agent(name="Nutrition Assistant", ...)
- Final output (str):
    - Banana: 89 kcal per 100 g. About 105 kcal for a medium banana (~118 g).
    - Apple: 52 kcal per 100 g. About 95 kcal for a medium apple (~182 g).
    
    Total for one banana + one apple (roughly 300 g total): ~200 kcal.
- 7 new item(s)
- 2 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


In [16]:
with trace("Nutrition Assistant with MCP "):
    result = await Runner.run(
        agent, "How many calories are in an english breakfast?"
    )
    print(result.final_output)

A full English breakfast can vary a lot, but a typical serving is roughly:

- Eggs (2 fried): ~180 kcal
- Bacon (2 slices): ~90–120 kcal
- Sausage (1 large): ~180–240 kcal
- Baked beans (1/2 cup): ~140 kcal
- Grilled tomato (1): ~20 kcal
- Sautéed mushrooms (1/2 cup): ~40 kcal
- Black pudding (1 slice): ~120–180 kcal
- Toast (2 slices) with butter: ~170–240 kcal

Estimated total: about 800–1,200 kcal. If you skip items (e.g., only eggs + toast) or cook with less oil, the total will be lower; adding extras (more sausage, hash browns, extra butter) will push it higher.


In [17]:
with trace("Nutrition Assistant with MCP "):
    result = await Runner.run(
        calorie_agent_with_search, "How many calories are in an english breakfast?"
    )
    print(result.final_output)

Approximately 860 calories per serving.

Ingredients (per BBC-style Full English, 1 serving) and estimated calories:
- Sausages: 2 x 70 g = 140 g → about 322 kcal
- Bacon: 3 rashers ~ 30 g → about 122 kcal
- Mushrooms: 2 flat mushrooms ~ 70 g → about 15 kcal
- Tomatoes: 2 medium ~ 240 g → about 43 kcal
- Black pudding: 60 g → about 227 kcal
- Egg: 1 large ~ 50 g → about 72 kcal
- Slice of bread: ~ 25 g → about 60 kcal

Total ≈ 860 kcal (varies with brands and exact weights).
